In [ ]:
#A popular library for testing the strength of cryptosystems
!python -m pip install ciphey --upgrade

In [2]:
# !pip install urllib3
# !pip install memory_profiler

In [3]:
import random

#Error checking code that ensures the data is not tampered with/valid
class ErrorChecking:
    def __init__(self, string):
        #the string to be checked
        self.string = string
        
    def split_by_n(self, seq, n):
        """
        Splits an integer into groups of n i.e. 1234, 2 will be grouped by [1,2], [3,4]
        Inputs: int, int: a sequence of numbers, the group size
        Outputs: lst: a list of numbers grouped by n
        """
        seq = str(seq)
        while seq:
            yield int(seq[:n])
            seq = seq[n:]
    
    def encode_string(self):
        """
        Converts a string to its binary representative
        Input: None (uses string from __init__)
        Output: lst: the binary representation of the input
        """
        return [bin(ord(x))[2:].zfill(8) for x in self.string]

    def decode_string(self, binary):
        """
        Converts binary to its ASCII representative
        Input: str: Takes in a binary as str
        Output: str: The ASCII representative of the binary 
        """
        if type(binary) == int:
            binary = str(binary)
        return ''.join([chr(int(x, 2)) for x in binary])
    
    def generate_hamming_code(self, bits):
        """
        Generates a hamming code and appends a parity bit at each position of power of 2
        Input: bin: A binary string that represents the initial input
        Ouptut: bin: A binary string with calculated parity bits
        """
        final_string = ''
        for i in bits:
            final_string += i
        data=list(final_string)
        data.reverse()
        c,ch,j,r,h=0,0,0,0,[]

        #Get the number of powers 2^r present in the list
        while ((len(data)+r+1)>(pow(2,r))):
            r=r+1

        for i in range(0,(r+len(data))):
            p=(2**c)

            if(p==(i+1)):
                h.append(0)
                c=c+1

            else:
                h.append(int(data[j]))
                j=j+1

        #For each parity bit, do the XOR operation to get difference
        for parity in range(0,(len(h))):
            ph=(2**ch)
            if(ph==(parity+1)):
                startIndex=ph-1
                i=startIndex
                toXor=[]

                while(i<len(h)):
                    block=h[i:i+ph]
                    toXor.extend(block)
                    i+=2*ph

                for z in range(1,len(toXor)):
                    h[startIndex]=h[startIndex]^toXor[z]
                ch+=1

        h.reverse()
        return (int(''.join(map(str, h))))
    
    def simulate_bit_error(self, binary):
        """
        Simulate an error in data transfer by randomly changing the value of a bit to its complement.
        Input: str: A string of binary numbers, the number of bits to change (corresponds to increase in data speed)
        Output: str: a modified binary
        """
        random_value = random.randint(0, len(str(binary)))
        buffer = [int(x) for x in str(binary)] 
        if buffer[random_value] == 1:
            buffer[random_value] = 0
        else:
            buffer[random_value] = 1
            
        return int(''.join(map(str, buffer)))

    def hamming(self, s1, s2):
        """
        Calculates the hamming distance and gets the erreneous letters and their position
        Input: str, str: Takes in two strings, the original and the decrypted one
        Output: hashmap, int: {position: {erreneous letters}}, hamming distance
        """
        unmatched = {}
        distance = 0
        if len(s1)!=len(s2):
            print("The two strings are not equal")
        else:
            for x,(i,j) in enumerate(zip(s1,s2)):
                if i!=j:
                    unmatched.update({x: {i, j}})
                    distance += 1
        
        return unmatched, distance
    
    def detect_errors(self, bit_error):
        """
        Detects the erreneous bit(s) and attempts to correct it.
        Input: bin: a Hamming code that has the parity bit attached
        Output: bool, bin: Returns whether the binary is tampered with and the recovered binary
        """
        if type(bit_error) == int:
            data = [int(x) for x in str(bit_error)]
        else:
            data=list(bit_error)
        data.reverse()
        c,ch,j,r,error,h,parity_list,h_copy=0,0,0,0,0,[],[],[]

        #Make 2 arrays of the data, get counts of powers 2^n in the list
        for k in range(0,len(data)):
            p=(2**c)
            h.append(int(data[k]))
            h_copy.append(data[k])
            if(p==(k+1)):
                c=c+1

        #Loop through each parity in the list
        for parity in range(0,(len(h))):
            ph=(2**ch)
            if(ph==(parity+1)):                
                startIndex=ph-1
                i=startIndex
                toXor=[]

                #Move through each block and make a list of blocks
                while(i<len(h)):
                    block=h[i:i+ph]
                    toXor.extend(block)
                    i+=2*ph

                #XOR Operation to get the different bits
                for z in range(1,len(toXor)):
                    h[startIndex]=h[startIndex]^toXor[z]
                parity_list.append(h[parity])
                ch+=1
        #Reverse back the list
        parity_list.reverse()
        #Get the number of erreneous bits
        error=sum(int(parity_list) * (2 ** i) for i, parity_list in enumerate(parity_list[::-1]))

        #If no error
        if((error)==0):
            print('\nThere is no error in the hamming code received')
            return 0

        #If both arrays (copied and modified) have the same length
        elif((error)>=len(h_copy)):
            print('Error cannot be detected')
            return None

        else:
            #Get the error's position and correct it by substitution
            print('Error is in',error,'bit')

            if(h_copy[error-1]=='0'):
                h_copy[error-1]='1'

            elif(h_copy[error-1]=='1'):
                h_copy[error-1]='0'
                print('After correction hamming code is:- ')
            h_copy.reverse()
            return (int(''.join(map(str, h_copy))))  
        
    def remove_parity_bits(self, binary):
        """
        Decodes the Hamming code by removing the parity bits initially appended
        Input: lst: A binary list with parity bits
        Output: lst: A list of binaries free of parity bits
        """
#         binary = ''.join(binary)
        n = len(binary)
        counter = 0
        position = []

        while True:
            pos = 2**counter
            if pos >= n:
                break
            counter += 1
            position.append(pos)
            
        for i in position:
            del binary[i]
        return binary

In [4]:
import random

class Encryptor:
    def __init__(self, string, public_key=(0, 0)):
        self.string = string
        self.public_key = public_key
        self.p, self.q = self.public_key
        
    def is_prime(self, num):
        """
        Objective: A function that checks whether a number is a prime number. 
        It implements a brute force approach for checking for all the factors of a number.
        
        Input: int: Takes in an integer which is a real number.
        Output: int: Returns a number; either 1 or -1 
        """
        if num > 1:
            for i in range(2, num):
                if (num % i) == 0:
                    return -1
                    break
                else:
                    return 1
        else:
            return -1


    def GCD(self, x, y):
        """
        Computes the Greatest Common Divosor of a pair of numbers.
        Input: int: A pair of integers that are real numbers
        Output: int: Returns a number that is the GCD of the two input.
        """
        while y != 0:
            c = x % y
            x = y
            y = c
        return x

    def mod_inverse(self, a, m):
        """
        Computes the inverse modulo (modular multiplicative inverse) of an integer.
        Input: int: a set of two numbers
        Ouptut: int/None: an integer representing the mod inverse of a number, None if not found
        """
        for x in range(1, m):
            if (a * x) % m == 1:
                return x
        return None

    def coprimes(self, number):
        """
        A function that computes the coprimes of a number.
        Input: 
        Output: Returns an array of the coprimes of a number.
        """
        l = []
        phi=(self.p-1)*(self.q-1)
        for x in range(2, number):
            if self.GCD(number, x) == 1 and self.mod_inverse(x, phi) != None:
                l.append(x)
        for x in l:
            if x == self.mod_inverse(x, phi):
                l.remove(x)
        return l

    def modular(self, x,y):
        """
        Computes the modular of two numbers.
        Input: int: A pair of numbers, the first one being the numerator
        Output: int: Returns a number that is the remainder of division of the inputs
        """
        if(x<y):
            return y
        else:
            c=x%y
            return c
        
    def encode(self, string):
        """
        Encodes a string from ASCII to binary
        Input: str: An arbitrary string
        Ouput: lst: a list of binary numbers
        """
        if type(string) == str:
            return [bin(ord(x))[2:].zfill(8) for x in string]
        else:
            return None
    
    def decode(self, binary):
        """
        Decodes a string from binary to ASCII
        Input: str: A list of binary numbers
        Ouput: lst: a string pertaining to the original binary
        """
        if type(string) == str:
            return ''.join([chr(int(x, 2)) for x in binary])
        else:
            return None
        
    def get_keys(self):
        """
        Computes the value of N, phi and randomly picks the coprimes.
        Input: int, int: prime number 1 and prime number 2
        Output: int: the encryption and decryption keys
        """    
        n = self.p * self.q
#         print("n = p * q = " + str(n))

        phi=(self.p-1)*(self.q-1)
#         print("Euler's function : " + str(phi) + "\n")

        encryption_key = random.choice(self.coprimes(phi))
        decryption_key = self.mod_inverse(encryption_key, phi)
        
        return encryption_key, decryption_key

    def encryption(self, message, encryption_string):
        """
        Encrypts the string using the power of prime factors and substitution.
        Input: str, int: takes in a string of text which will be the plaintext and the key
        Output: str: An encrypted string
        """
        cipher=""
        n = self.p * self.q
        for x in list(message):
            c = self.modular(ord(x)**encryption_string, n)
            cipher+=(chr(c))
        return cipher
    
    def decryption(self, decrypted_string, decryption_key):
        """
        Takes in an encrypted string and decrypts it using the private key.
        Input: str, int: takes in a decrypted string as a string
        Output: str: The original message
        """
        plain=""
        n = self.p * self.q
        for x in list(decrypted_string):
            c = self.modular(ord(x)**decryption_key, n)
            plain+=(chr(c))
        return plain

In [6]:
if __name__=="__main__":
    print("\n1. The bigger the prime number, the better")
    print("2. The prime numbers should be non-repeating and of the same length.")
    print("Example: (41,37) ")

    prime_number_1 = int(input('Enter the first prime number: '))
    prime_number_2 = int(input('Enter the second prime number: '))
    
    s = input("Enter a text to encrypt: ")
    encrypt = Encryptor(s, (prime_number_1, prime_number_2))
    if encrypt.is_prime(prime_number_1) == -1 or encrypt.is_prime(prime_number_2) == -1:
         raise SystemExit("Please select a PRIME NUMBER !!!")

    print("\nYour choices are : p=" + str(prime_number_1) + ", q=" + str(prime_number_2))
    encyption_key, decryption_key = encrypt.get_keys()

    print("\nYour public key is (e = " + str(encyption_key) + ")")
    print("Your private key is (d = " + str(decryption_key) + ")")
    print("\nPlain message: " + s)

    if type(s) == str:
        
        enc = encrypt.encryption(s, encyption_key)
        print("Encrypted message: " + str(enc))

        dec = encrypt.decryption(enc, decryption_key)
        print("Decrypted message: " + dec + "\n")

    else:
        print("Invalid choice")


1. The bigger the prime number, the better
2. The prime numbers should be non-repeating and of the same length.
Example: (41,37) 


Enter the first prime number:  41
Enter the second prime number:  37
Enter a text to encrypt:  hey



Your choices are : p=41, q=37

Your public key is (e = 1343)
Your private key is (d = 1247)

Plain message: hey
Encrypted message: ԃɛы
Decrypted message: hey



In [7]:
import urllib.request

target_url = 'https://raw.githubusercontent.com/mgeisler/lipsum/master/src/lorem-ipsum.txt'

txt_file = [line.decode('utf-8').split() for line in urllib.request.urlopen(target_url)]
words = [item for sublist in txt_file for item in sublist]
string = ' '.join(words)

print("Initial string: " + string)

if __name__=='__main__':
    code = ErrorChecking(string)
    word_in_binary = code.encode_string()
    hamming_code = code.generate_hamming_code(word_in_binary)

    if code.detect_errors(hamming_code) == 0:
        final_string = string
        print("\nCorrect string: " + code.decode_string(word_in_binary))
    elif code.detect_errors(hamming_code) == None:
        pass
    else:
        recovered_binary = code.detect_errors(hamming_code)
        recovered_binary = list(map(str, list(code.split_by_n(recovered_binary, 8)))) 
        recovered_binary = code.remove_parity_bits(recovered_binary)
        final_string = code.decode_string(recovered_binary)
        print("Recovered string: " + str(recovered_binary))

    #Checking for correctness between original phrase and decrypted string
    mismatch, distance = code.hamming(string, final_string)
    if distance == 0:
        print("\nHamming distance: " + str(distance))
        print("The two phrases match!")
    else:
        print("Hamming distance: " + str(distance))

Initial string: Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

There is no error in the hamming code received

Correct string: Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

Hamming distance: 0
The tw

In [8]:
import urllib.request

target_url = 'https://raw.githubusercontent.com/mgeisler/lipsum/master/src/lorem-ipsum.txt'

txt_file = [line.decode('utf-8').split() for line in urllib.request.urlopen(target_url)]
words = [item for sublist in txt_file for item in sublist]
string = ' '.join(words)

print("Initial string: " + string)

if __name__=='__main__':
    code = ErrorChecking(string)
    word_in_binary = code.encode_string()
    hamming_code = code.generate_hamming_code(word_in_binary)
    changed_hamming_code = code.simulate_bit_error(hamming_code)
    if(len(str(hamming_code)) == len(str(changed_hamming_code))):
        print("\nSimulated bit error has the same length.")

    if code.detect_errors(changed_hamming_code) == 0:
        final_string = string
        print("\nCorrect string: " + code.decode_string(word_in_binary))
    elif code.detect_errors(changed_hamming_code) == None:
        pass
    else:
        recovered_binary = code.detect_errors(changed_hamming_code)
        recovered_binary = list(map(str, list(code.split_by_n(recovered_binary, 8)))) 
        print("\nAttempting to recover lost or malformed bits")
        recovered_binary = code.remove_parity_bits(recovered_binary)
        final_string = code.decode_string(recovered_binary)
        print("\nCorrect string: " + code.decode_string(recovered_binary))


    #Checking for correctness between original phrase and decrypted string
    mismatch, distance = code.hamming(string, final_string)

Initial string: Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

Simulated bit error has the same length.
Error is in 630 bit
Error is in 630 bit
Error is in 630 bit

Attempting to recover lost or malformed bits

­í.-Í-¤BW7BÓ{ÕªNä@æÒ@ÂÚÊèX@ÆÞÜæÊÆèÊèä@ÂÈÒàÒæÆÒÜÎ@ÊØÒèX@æÊÈ@ÈÞ@ÊÒêæÚÞ@èÊÚàÞä@ÒÜÆÒÈÒÈêÜè@êè@ØÂÄÞäÊ@Êè@ÈÞØÞäÊ@ÚÂÎÜÂ@ÂØÒâêÂ\@ªè@ÊÜÒÚ@ÂÈ@ÒÜÒÚ@ìÊÜÒÂÚX@âêÒæ@ÜÞæèäêÈ@ÊðÊäÆÒèÂèÒÞÜ@êØØÂÚÆÞ@ØÂÄÞäÒæ@nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehederit in voluptate velit esse cillum dolore eu fugiat07:¶6080¹4°º:¹"¼1²¸:2º¹9´·:7±±°²±°º1º¸4²0º0¾77·897´²2·:

### Combined

In [9]:
if __name__=="__main__":
    print("\n1. The bigger the prime number, the better")
    print("2. The prime numbers should be non-repeating and of the same length.")
    print("Example: (41,37) ")

    prime_number_1 = int(input('Enter the first prime number: '))
    prime_number_2 = int(input('Enter the second prime number: '))
    
    s = input("Enter a text to encrypt: ")
    encrypt = Encryptor(s, (prime_number_1, prime_number_2))
    if encrypt.is_prime(prime_number_1) == -1 or encrypt.is_prime(prime_number_2) == -1:
         raise SystemExit("Please select a PRIME NUMBER !!!")

    print("\nYour choices are : p=" + str(prime_number_1) + ", q=" + str(prime_number_2))
    encyption_key, decryption_key = encrypt.get_keys()

    print("\nYour public key is (e = " + str(encyption_key) + ")")
    print("Your private key is (d = " + str(decryption_key) + ")")
    print("\nPlain message: " + s)

    if type(s) == str:
        # Create use the method instance - encryption to encrypt the message with the key
        enc = encrypt.encryption(s, encyption_key)
        print("Encrypted message: " + str(enc))
        
        code = ErrorChecking(s)
        word_in_binary = code.encode_string()
        hamming_code = code.generate_hamming_code(word_in_binary)
        print("Hamming code generated: " + str(hamming_code))

        changed_hamming_code = code.simulate_bit_error(hamming_code)

        if code.detect_errors(changed_hamming_code) == 0:
            final_string = string
            print("Correct string: " + code.decode_string(word_in_binary))
        elif code.detect_errors(changed_hamming_code) == None:
            pass
        else:
            recovered_binary = code.detect_errors(changed_hamming_code)
            recovered_binary = list(map(str, list(code.split_by_n(recovered_binary, 8)))) 
            final_string = code.decode_string(recovered_binary)
            print("Recovered string: " + code.decode_string(recovered_binary))
    
        #Checking for correctness between original phrase and decrypted string
        mismatch, distance = code.hamming(string, final_string)

        dec = encrypt.decryption(enc, decryption_key)
        print("Decrypted message: " + dec + "\n")


1. The bigger the prime number, the better
2. The prime numbers should be non-repeating and of the same length.
Example: (41,37) 


Enter the first prime number:  41
Enter the second prime number:  37
Enter a text to encrypt:  hey



Your choices are : p=41, q=37

Your public key is (e = 61)
Your private key is (d = 661)

Plain message: hey
Encrypted message: ՜һy
Hamming code generated: 1101000011001101011101001111
Error cannot be detected
Error cannot be detected
The two strings are not equal
Decrypted message: hey



### Test cases

In [84]:
#Encryption Test Cases

s1 = 'encryption' #normal input
s2 = '' #empty string
s3 = 2 #integer
s4 = 'fnsaknfdasnfjaksndfjkasndfjnfjqenrfqkjenrfjkwenrfjkwenrewjkrnfwekjrrfejrnfenfkenrlknwklefnlwknefwf' # really long string
s5 = ';:#@%$%&%*^&(^)' #symbols
s6 = '😊' #emoji (non UTF-8)
s7 = '123456' #numbers that are strings

if __name__=="__main__":
    prime_number_1 = 41
    prime_number_2 = 37
    
    try:
        s1 = str(s1)
        s2 = str(s2)
        s3 = str(s3)
        s4 = str(s4)
        s5 = str(s5)
        s6 = str(s6)
        s7 = str(s7)
    except ValueError:
        print('Input not a string')

    encrypt = Encryptor(s1, (prime_number_1, prime_number_2))
    encrypt2 = Encryptor(s2, (prime_number_1, prime_number_2))
    encrypt3 = Encryptor(s3, (prime_number_1, prime_number_2))
    encrypt4 = Encryptor(s4, (prime_number_1, prime_number_2))
    encrypt5 = Encryptor(s5, (prime_number_1, prime_number_2))
    encrypt6 = Encryptor(s6, (prime_number_1, prime_number_2))
    encrypt7 = Encryptor(s7, (prime_number_1, prime_number_2))
    encyption_key, decryption_key = encrypt.get_keys()

    if type(s1 and s2 and s3 and s4 and s5 and s6 and s7) == str:
        
        enc = encrypt.encryption(s1, encyption_key)
        enc2 = encrypt2.encryption(s2, encyption_key)
        enc3 = encrypt3.encryption(s3, encyption_key)
        enc4 = encrypt4.encryption(s4, encyption_key)
        enc5 = encrypt5.encryption(s5, encyption_key)
        enc6 = encrypt6.encryption(s6, encyption_key)
        enc7 = encrypt7.encryption(s7, encyption_key)
        print("Encrypted message 1: " + str(enc))
        print("Encrypted message 2: " + str(enc2))
        print("Encrypted message 3: " + str(enc3))
        print("Encrypted message 4: " + str(enc4))
        print("Encrypted message 5: " + str(enc5))
        print("Encrypted message 6: " + str(enc6))
        print("Encrypted message 7: " + str(enc7) + "\n")

        dec = encrypt.decryption(enc, decryption_key)
        dec2 = encrypt2.decryption(enc2, decryption_key)
        dec3 = encrypt3.decryption(enc3, decryption_key)
        dec4 = encrypt4.decryption(enc4, decryption_key)
        dec5 = encrypt5.decryption(enc5, decryption_key)
        dec6 = encrypt6.decryption(enc6, decryption_key)
        dec7 = encrypt7.decryption(enc7, decryption_key)
        print("Decrypted message 1: " + dec)
        print("Decrypted message 2: " + dec2)
        print("Decrypted message 3: " + dec3)
        print("Decrypted message 4: " + dec4)
        print("Decrypted message 5: " + dec5)
        print("Decrypted message 6: " + dec6)
        print("Decrypted message 7: " + dec7)

    else:
        print("Invalid choice")

n = p * q = 1517
Euler's function : 1440

Encrypted message 1: 0ҟø՚˸äȆҟ
Encrypted message 2: ǃˏ0ȋƶ0Ԩ˸ˏȊǃˏ։ȆԨˏȊˏēäҟƶǞˏŖˏ։ƶ՚֭Ԭ˸0ˏȊˏ˛Ȇ˟äƶ˟ˏŖˏԬ0՚˸ɓȆ˟Ԭ˸ˏȊˏǞǣŖǞˬˏŖˏԬ՚՚ˏȊʇ
Encrypted message 3: ǣ
Encrypted message 4: ɓҟԨԬďҟɓ֭ԬԨҟɓ̿ԬďԨҟ֭ɓ̿ďԬԨҟ֭ɓ̿ҟɓ̿ȋ0ҟɓȋď̿0ҟɓ̿ďΫ0ҟɓ̿ďΫ0ҟ0Ϋ̿ďҟɓΫ0ď̿ɓ0̿ҟɓ0ҟɓď0ҟēďҟΫďē0ɓҟēΫďҟ0ɓΫɓ
Encrypted message 5: ͆ȊE։%ʾ%ȇ%ȿ8ȇϗ8ղ
Encrypted message 6: ͗
Encrypted message 7: ȍǣˬЙξ

Decrypted message 1: encryption
Decrypted message 2: {"request":{"@os":"linux","@updater":"chromium","acceptformat":"crx2,crx3","app":[
Decrypted message 3: 2
Decrypted message 4: fnsaknfdasnfjaksndfjkasndfjnfjqenrfqkjenrfjkwenrfjkwenrewjkrnfwekjrrfejrnfenfkenrlknwklefnlwknefwf
Decrypted message 5: ;:#@%$%&%*^&(^)
Decrypted message 6: ц
Decrypted message 7: 123456


### Appendix

In [ ]:
#import dependancies for testing the strength of the cryptosystem
from ciphey import decrypt
from ciphey.iface import Config

def test_encryptor():
    """
    Test the encryptor's strength by trying to crack common cryptographic algorithms
    Input: None (string should be added)
    Output: the decrypted string
    """
    res = decrypt(
        Config().library_default().complete_config(),
        '0ҟø՚˸äȆҟ',
    )
    return res

answer = test_affine()
print(answer)

In [107]:
# Driver code for importing the text file used in the demonstration

target_url = 'https://raw.githubusercontent.com/mgeisler/lipsum/master/src/lorem-ipsum.txt'

import urllib.request

txt_file = [line.decode('utf-8').split() for line in urllib.request.urlopen(target_url)]
words = [item for sublist in txt_file for item in sublist]
string = ' '.join(words)
print(string)

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.


In [ ]:
def calcRedundantBits(m): 
    """
    A function that calculates the parity bit based on the bit and its position
    Input: int: the length of the binary string
    Output: bin: The result of the parity bit 
    """
    for i in range(m): 
        if(2**i >= m + i + 1): 
            return i 

def posRedundantBits(data, r):
    """
    Sets the position of the parity bits
    Input: bin, int: a binary string, an integer of the calculated parities
    Ouput: lst: A list of all the parity bits
    """
    j = 0
    k = 1
    m = len(data) 
    res = '' 
    for i in range(1, m + r+1): 
        if(i == 2**j): 
            res = res + '0'
            j += 1
        else: 
            res = res + data[-1 * k] 
            k += 1
    return res[::-1] 

bits = ''.join(recovered_binary)
m = len(bits) 
r = calcRedundantBits(m) 
posi = posRedundantBits(bits, r)

word = ''.join(word_in_binary)
print(len(word))
print(len(posi))

In [106]:
def hamming_distance(s1,s2):
    """
    Calculates the hamming distance and gets the erreneous letters and their position
    Input: str, str: Takes in two strings, the original and the decrypted one
    Output: hashmap, int: {position: {erreneous letters}}, hamming distance
    """
    unmatched = {}
    distance = 0
    if len(s1)!=len(s2):
        print("String are not equal")
    else:
        for x,(i,j) in enumerate(zip(s1,s2)):
            if i!=j:
                unmatched.update({x: {i, j}})
                distance += 1
    return unmatched, distance

s1="mark gacoka"
s2="marr gakoca"
print(hamming_distance(s1,s2))

({3: {'r', 'k'}, 7: {'k', 'c'}, 9: {'k', 'c'}}, 3)


In [86]:
import time

def crytanalysis_encrypt():
    prime_number_1 = 41
    prime_number_2 = 37
    start = time.time()
    string = 'a' * 1000
    for i in range(1000):
        encrypt = Encryptor(string, (prime_number_1, prime_number_2))
        enc = encrypt.encryption(string, encyption_key)
    end = time.time()
    return(end - start)

encryption_time = crytanalysis_encrypt()
print(encryption_time)

16.877848625183105


In [102]:
import time

def crytanalysis_decrypt():
    prime_number_1 = 41
    prime_number_2 = 37
    start = time.time()
    for i in range(1000):
        encrypt = Encryptor(string, (prime_number_1, prime_number_2))
        encyption_key, decryption_key = encrypt.get_keys()
        dec = encrypt.decryption(enc, decryption_key)
    end = time.time()
    return(end - start)

decryption_time = crytanalysis_decrypt()
print(decryption_time)

54.06207633018494


In [104]:
%load_ext memory_profiler
%memit
def crytanalysis_decrypt():
    prime_number_1 = 41
    prime_number_2 = 37
    start = time.time()
    for i in range(1000):
        encrypt = Encryptor(string, (prime_number_1, prime_number_2))
        encyption_key, decryption_key = encrypt.get_keys()
        dec = encrypt.decryption(enc, decryption_key)
    end = time.time()
    return(end - start)

decryption_time = crytanalysis_decrypt()

peak memory: 43.13 MiB, increment: 0.13 MiB


In [105]:
%load_ext memory_profiler
%memit
def crytanalysis_encrypt():
    prime_number_1 = 41
    prime_number_2 = 37
    start = time.time()
    string = 'a' * 1000
    for i in range(1000):
        encrypt = Encryptor(string, (prime_number_1, prime_number_2))
        enc = encrypt.encryption(string, encyption_key)
    end = time.time()
    return(end - start)

encryption_time = crytanalysis_encrypt()

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
peak memory: 44.86 MiB, increment: 0.00 MiB
